# Models to predict ratings

In [20]:
import pandas as pd
import numpy as np

friends = pd.read_csv('transcripts and metadata/friends_mastersheet.csv', index_col=0)

In [21]:
friends.head()

Year_of_prod  Season  Episode Number  \
0          1994       1               1   
1          1994       1               2   
2          1994       1               3   
3          1994       1               4   
4          1994       1               5   

                                     Episode_Title  Duration  \
0  The One Where Monica Gets a Roommate: The Pilot        22   
1             The One with the Sonogram at the End        22   
2                           The One with the Thumb        22   
3               The One with George Stephanopoulos        22   
4   The One with the East German Laundry Detergent        22   

                                             Summary       Director  Stars  \
0  Monica and the gang introduce Rachel to the "r...  James Burrows    8.3   
1  Ross finds out his ex-wife is pregnant. Rachel...  James Burrows    8.1   
2  Monica becomes irritated when everyone likes h...  James Burrows    8.2   
3  Joey and Chandler take Ross to a hockey game t...  James Burrows    8.1   
4  Eager to spend time with Rachel, Ross pretends...  Pamela Fryman    8.5   

   Votes  match  ...    Phoebe      Ross    Rachel     Carol     Susan  \
0   7440     11  ...  0.061248  0.144114  0.155643  0.000000  0.000000   
1   4888     12  ...  0.061500  0.248342  0.165269  0.088068  0.044696   
2   4605     13  ...  0.145415  0.125624  0.102383  0.000000  0.000000   
3   4468     14  ...  0.111161  0.151849  0.162042  0.000000  0.000000   
4   4438     15  ...  0.095007  0.150061  0.141764  0.000000  0.000000   

     Janice  Mike  Gunther  Ben  Emily  
0  0.000000   0.0      0.0  0.0    0.0  
1  0.000000   0.0      0.0  0.0    0.0  
2  0.000000   0.0      0.0  0.0    0.0  
3  0.000000   0.0      0.0  0.0    0.0  
4  0.047483   0.0      0.0  0.0    0.0  

[5 rows x 26 columns]

In [22]:
resultlist = [['model', 'LOOCV mean score', 'std']]
books = pd.read_csv('books/friends_books_topics_model.csv', index_col=0)
sb = pd.read_csv('scriptbase/friends_scriptbase_topics.csv', index_col=0)
friends_no_topics = pd.read_csv('transcripts and metadata/friends_mastersheet.csv', index_col=0)

## Baselines

### dummy baseline

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

cv = LeaveOneOut()

X = friends.text
y = friends.Stars

vect = TfidfVectorizer(max_features=5000)
reg = DummyRegressor(strategy="mean")
pipe = make_pipeline(vect, reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

resultlist.append(['dummy_baseline', np.mean(scores), np.std(scores)])

print(np.mean(scores), np.std(scores))

-0.3113667508113955 0.24655333378118335


### BOW baseline

In [24]:
from sklearn.svm import SVR

cv = LeaveOneOut()

X = friends.text
y = friends.Stars

vect = TfidfVectorizer(max_features=5000)
reg = SVR()
pipe = make_pipeline(vect, reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

resultlist.append(['BOW_baseline', np.mean(scores), np.std(scores)])

print(np.mean(scores), np.std(scores))

-0.31038842338807143 0.2511842652329307


In [25]:
model = pipe.fit(X, y)

### No topics baseline

In [26]:
lst = ['Gary Halvorson',        
'Kevin Bright',          
'Michael Lembeck',       
'James Burrows',           
'Gail Mancuso',           
'Peter Bonerz',            
'David Schwimmer',         
'Ben Weiss']

def direr(wow):
    if wow not in lst:
        wow = wow.replace(wow,'Other')
    return wow

In [27]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = friends_no_topics.drop(columns=['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = friends_no_topics.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))
resultlist.append(['no_topic_baseline', np.mean(scores), np.std(scores)])

model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3152065310522481 0.24494497880925878


## Models all features

### Books corpus

In [28]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = books.drop(columns=['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_all_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# # feature_names = model[:-1].get_feature_names_out()

# # X_preprocessed = pd.DataFrame(
# #     model[:-1].transform(X), columns=feature_names
# # )

# # coefs = pd.DataFrame(
# #     model[-1].coef_* X_preprocessed.std(axis=0),
# #     columns=["Coefficients"],
# #     index=feature_names,
# # )

# # coefs["Coefficients"].sort_values(ascending=False)

-0.31440869871969496 0.2531891965351134


### scriptbase

In [29]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

X = sb.drop(columns=['Summary', 'Episode_Title', 'Year_of_prod','Votes', 'text', 'match', 'len in words', 'len_prep'])
X.Director = X.Director.apply(direr)
y = sb.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sc_all_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.2634689459745494 0.20842686080328332


## models 50

### books

In [30]:
books_best_50 = ['t83', 't188', 't274', 't230', 't273', 't204', 't196',
       't176', 't7', 't120', 't151', 't258', 't139', 't70', 't110', 't97',
       't235', 't282', 't104', 't62', 't245', 't80', 't171', 't115', 't270',
       't109', 't150', 't269', 't158', 't203', 't59', 't295', 't175', 't51',
       't166', 't113', 't285', 't299', 't286', 't88', 't127', 't69', 't222',
       't272', 't34', 't58', 't153', 't271', 't259', 't73']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in books_best_50]

In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary',  'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_50_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.2992068909287241 0.25312880887507766


### scriptbase

In [32]:
sb_best_50 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239',
       't205', 't26', 't86', 't7', 't119', 't296', 't67', 't265', 't152',
       't128', 't72', 't250', 't242', 't287', 't190', 't57', 't44', 't17',
       't132', 't189', 't240', 't193', 't21', 't138', 't130']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_50]

In [33]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_50_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.2765580952051211 0.22592420845009073


## models 25

### books

In [34]:
books_best_25 = ['t83', 't188', 't274', 't230', 't273', 't204', 't196',
't176', 't7', 't120', 't151', 't258', 't139', 't70', 't110', 't97',
't235', 't282', 't104', 't62', 't245', 't80', 't171', 't115', 't270']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in books_best_25]

In [35]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_25_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.3020732451010574 0.23819317506814044


### scriptbase

In [36]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25]

In [37]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_25_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.28001743143273483 0.2195431482604701


## Models 10

### books

In [38]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25[:10]]

In [39]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = books.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['books_10_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.31234145291979715 0.2573858774135108


### scriptbase

In [40]:
sb_best_25 = ['t134', 't103', 't12', 't194', 't276', 't35', 't180',
       't222', 't32', 't191', 't299', 't64', 't202', 't288', 't275', 't260',
       't84', 't182', 't165', 't210', 't90', 't204', 't255', 't217', 't239']

to_drop = [f't{i}' for i in range(0,300) if f't{i}' not in sb_best_25[:10]]

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

columns = ['Summary', 'Episode_Title', 'Stars', 'Votes', 'text', 'match', 'len in words', 'len_prep'] + to_drop

X = sb.drop(columns=columns)
X.Director = X.Director.apply(direr)
y = books.Stars

reg = SVR()
hot = OneHotEncoder()
scal = StandardScaler()

categorical_features = X.select_dtypes(include="object").columns
integer_features = X.select_dtypes(exclude="object").columns

prep = make_column_transformer((hot, categorical_features),
                                (scal, integer_features))

pipe = make_pipeline(prep,reg)

scores = cross_val_score(pipe, X, y, cv=cv, scoring = 'neg_root_mean_squared_error')

print(np.mean(scores), np.std(scores))

resultlist.append(['sb_10_topics', np.mean(scores), np.std(scores)])

# model = pipe.fit(X, y)

# feature_names = model[:-1].get_feature_names_out()

# X_preprocessed = pd.DataFrame(
#     model[:-1].transform(X), columns=feature_names
# )

# coefs = pd.DataFrame(
#     model[-1].coef_* X_preprocessed.std(axis=0),
#     columns=["Coefficients"],
#     index=feature_names,
# )

# coefs["Coefficients"].sort_values(ascending=False)

-0.29508784576346614 0.24615329370423772


In [41]:
resultlist

[['model', 'LOOCV mean score', 'std'],
 ['dummy_baseline', -0.3113667508113955, 0.24655333378118335],
 ['BOW_baseline', -0.31038842338807143, 0.2511842652329307],
 ['no_topic_baseline', -0.3152065310522481, 0.24494497880925878],
 ['books_all_topics', -0.31440869871969496, 0.2531891965351134],
 ['sc_all_topics', -0.2634689459745494, 0.20842686080328332],
 ['books_50_topics', -0.2992068909287241, 0.25312880887507766],
 ['sb_50_topics', -0.2765580952051211, 0.22592420845009073],
 ['books_25_topics', -0.3020732451010574, 0.23819317506814044],
 ['sb_25_topics', -0.28001743143273483, 0.2195431482604701],
 ['books_10_topics', -0.31234145291979715, 0.2573858774135108],
 ['sb_10_topics', -0.29508784576346614, 0.24615329370423772]]